<a href="https://colab.research.google.com/github/wolfram-laube/mlpc-project_team-park/blob/wl/task_4/fastlane.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Preparation: Distiguish Environment and Establish Prerequisites

#### Setup Folder Structure

In [37]:
import os
import sys
import shutil

# Check if the environment is Google Colab
try:
    import google.colab
    IN_COLAB = True
except ImportError:
    IN_COLAB = False

if IN_COLAB:
    # If in Google Colab
    from google.colab import drive
    import gdown

    # Option 1: Download the file by its public link and expand it to the Colab runtime
    import urllib.request
    import zipfile

    zip_file_id = '1oI1EsH1krrEPbH9MSZRzLHu-_4p6-njR' # https://drive.google.com/file/d/1oI1EsH1krrEPbH9MSZRzLHu-_4p6-njR/view?usp=sharing
    annotation_file_id = '1xLxget7c5nCkwYt9Ru2RpYi5rMkk_pl0'  # https://drive.google.com/file/d/1xLxget7c5nCkwYt9Ru2RpYi5rMkk_pl0/view?usp=sharing
    scenes_file_id = '1xLgB7-cCz6nReyQbFJJcJGOUKCCbNhCG'  # https://drive.google.com/file/d/1xLgB7-cCz6nReyQbFJJcJGOUKCCbNhCG/view?usp=sharing

    zip_url = f'https://drive.google.com/uc?id={zip_file_id}'
    annotation_url = f'https://drive.google.com/uc?id={annotation_file_id}'
    scenes_url = f'https://drive.google.com/uc?id={scenes_file_id}'

    zip_path = '/content/training_data.zip'
    data_dir = '/content/dataset'
    scenes_dir = f'{data_dir}/scenes'
    words_dir = f'{data_dir}/words'
    scenes_wav_dir = f'{scenes_dir}/wav'
    scenes_npy_dir = f'{scenes_dir}/npy'

    # Download the ZIP file
    #urllib.request.urlretrieve(zip_url, zip_path)
    gdown.download(zip_url, zip_path, quiet=False)

    # Unzip the file
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(data_dir)

    print(f"Training data extracted to {data_dir}")

     # Create the 'scenes/wav' folder structure
    os.makedirs(scenes_wav_dir, exist_ok=True)
    os.makedirs(scenes_npy_dir, exist_ok=True)

    # Copy .wav files to 'scenes/wav'
    extracted_scenes_dir = os.path.join(data_dir, 'mlpc24_speech_commands', 'scenes')
    for root, dirs, files in os.walk(extracted_scenes_dir):
        for file in files:
            if file.endswith('.wav'):
                src_path = os.path.join(root, file)
                dst_path = os.path.join(scenes_wav_dir, file)
                shutil.copy(src_path, dst_path)

    print(f".wav files moved to {scenes_wav_dir}")

    # Download the CSV files into the data_dir
    annotation_orig_path = os.path.join(data_dir, 'development_scene_annotations.csv.orig') # Keep a bckup copy because it needs fixing
    annotation_path = os.path.join(data_dir, 'development_scene_annotations.csv')
    scenes_path = os.path.join(data_dir, 'development_scenes.csv')

    gdown.download(annotation_url, annotation_orig_path, quiet=False)
    gdown.download(annotation_url, annotation_path, quiet=False)
    gdown.download(scenes_url, scenes_path, quiet=False)

    print(f"CSV files downloaded to {scenes_dir}")

    # Option 2: Mount Google Drive and use the training data
    # Note this really takes some time for preprocessing file by file
    #drive.mount('/content/drive')
    #data_dir = '/content/drive/My Drive/Dropbox/public/mlpc/dataset'

    # Use this option to read from Google Drive instead
    #print(f"Using training data from {data_dir}")
else:
    # If on local machine
    data_dir = '../dataset'
    print(f"Using local training data from {data_dir}")

# Use the data_dir variable as the path to your training data

Downloading...
From (original): https://drive.google.com/uc?id=1oI1EsH1krrEPbH9MSZRzLHu-_4p6-njR
From (redirected): https://drive.google.com/uc?id=1oI1EsH1krrEPbH9MSZRzLHu-_4p6-njR&confirm=t&uuid=5a3a97f0-a86f-4c49-b394-a235da0bc999
To: /content/training_data.zip
100%|██████████| 305M/305M [00:01<00:00, 161MB/s]


Training data extracted to /content/dataset
.wav files moved to /content/dataset/scenes/wav


Downloading...
From: https://drive.google.com/uc?id=1xLxget7c5nCkwYt9Ru2RpYi5rMkk_pl0
To: /content/dataset/development_scene_annotations.csv.orig
100%|██████████| 70.4k/70.4k [00:00<00:00, 80.5MB/s]
Downloading...
From: https://drive.google.com/uc?id=1xLxget7c5nCkwYt9Ru2RpYi5rMkk_pl0
To: /content/dataset/development_scene_annotations.csv
100%|██████████| 70.4k/70.4k [00:00<00:00, 37.4MB/s]
Downloading...
From: https://drive.google.com/uc?id=1xLgB7-cCz6nReyQbFJJcJGOUKCCbNhCG
To: /content/dataset/development_scenes.csv
100%|██████████| 29.5k/29.5k [00:00<00:00, 30.6MB/s]

CSV files downloaded to /content/dataset/scenes


#### Fix Erreneous Metadata

##### Before

In [38]:
import pandas as pd

# Load the CSV files
scene_annotations_df = pd.read_csv(f'{data_dir}/development_scene_annotations.csv')
scenes_df = pd.read_csv(f'{data_dir}/development_scenes.csv')

# Check the head of the dataframes to understand their structure
print(scene_annotations_df.head())
print(scenes_df.head())

# Check the distribution of labels in the annotations CSV
label_distribution_annotations = scene_annotations_df['command'].value_counts()
print("Label Distribution in development_scene_annotations.csv:")
print(label_distribution_annotations)

# Check the distribution of speaker IDs in the scenes CSV
label_distribution_scenes = scenes_df['speaker_id'].value_counts()
print("Label Distribution in development_scenes.csv:")
print(label_distribution_scenes)


                        filename         command     start       end
0         2_speech_true_Ofen_aus        Ofen aus  11.25230  12.07747
1         3_speech_true_Radio_an  Staubsauger an  21.48040  23.18083
2         4_speech_true_Alarm_an        Alarm an  14.45720  16.08301
3        9_speech_true_Radio_aus  Staubsauger an   3.67909   5.63126
4  11_speech_false_Fernseher_aus  Staubsauger an  10.57850  11.67886
                        filename  speaker_id
0         2_speech_true_Ofen_aus         132
1         3_speech_true_Radio_an         132
2         4_speech_true_Alarm_an         132
3        9_speech_true_Radio_aus         132
4  11_speech_false_Fernseher_aus         132
Label Distribution in development_scene_annotations.csv:
command
Staubsauger an     288
Licht aus           77
Licht an            64
Alarm an            56
Fernseher an        56
Heizung an          55
Heizung aus         54
Radio aus           53
Radio an            52
Ofen aus            49
Alarm aus           4

##### Fix

In [39]:
import os
import re
import shutil
import pandas as pd

# Paths to the original and working copy files
original_file_path = f'{data_dir}/development_scene_annotations.csv.orig'
working_copy_path = f'{data_dir}/development_scene_annotations.csv.0'
corrected_file_path = f'{data_dir}/development_scene_annotations.csv'

# Step 1: Create a working copy of the original file
shutil.copy(original_file_path, working_copy_path)

# Step 2: Load the working copy into a DataFrame
df = pd.read_csv(working_copy_path)

# Define the pattern to parse the filename
filename_pattern = re.compile(r'(\d+)_speech_(true|false)_((?:[a-zA-ZäöüÄÖÜß]+_(?:an|aus)_?)+)', re.UNICODE)

# Function to parse filename and extract commands
def parse_filename(filename):
    match = filename_pattern.match(filename)
    if not match:
        return []

    commands_str = match.group(3)
    commands = commands_str.split('_')

    command_list = []
    for i in range(0, len(commands), 2):
        command_list.append(f"{commands[i]} {commands[i+1]}")

    return command_list

# Parse the commands from filenames and add to the DataFrame
df['parsed_commands'] = df['filename'].apply(parse_filename)

# Step 3: Group by filename and sort by start time
grouped = df.groupby('filename').apply(lambda x: x.sort_values(by='start')).reset_index(drop=True)

# Step 4: Assign the correct labels based on the order of commands in the filename
def assign_labels(group):
    commands = group['parsed_commands'].iloc[0]  # get the parsed commands from the first row
    group = group.reset_index(drop=True)
    for i in range(len(group)):
        if i < len(commands):
            group.at[i, 'command'] = commands[i]
        else:
            print(f"Warning: More segments than commands in {group['filename'].iloc[0]}")
    return group

# Apply the label assignment function
corrected_df = grouped.groupby('filename').apply(assign_labels).reset_index(drop=True)

# Drop the temporary column
corrected_df = corrected_df.drop(columns=['parsed_commands'])

# Step 5: Save the corrected DataFrame to a new CSV file
corrected_df.to_csv(corrected_file_path, index=False)

# Verify the saved corrections
print("Label corrections applied and saved successfully.")
print(corrected_df.head())


Label corrections applied and saved successfully.
                        filename       command     start       end
0    1003_speech_false_Licht_aus     Licht aus  12.20090  13.57599
1       1008_speech_true_Ofen_an       Ofen an   6.90112   8.52638
2      1010_speech_true_Radio_an      Radio an  13.03100  14.03146
3  1011_speech_true_Fernseher_an  Fernseher an  14.11030  15.36121
4   1012_speech_true_Heizung_aus   Heizung aus  11.20520  12.70590


##### After

In [40]:
import pandas as pd

# Load the CSV files
scene_annotations_df = pd.read_csv(f'{data_dir}/development_scene_annotations.csv')
scenes_df = pd.read_csv(f'{data_dir}/development_scenes.csv')

# Check the head of the dataframes to understand their structure
print(scene_annotations_df.head())
print(scenes_df.head())

# Check the distribution of labels in the annotations CSV
label_distribution_annotations = scene_annotations_df['command'].value_counts()
print("Label Distribution in development_scene_annotations.csv:")
print(label_distribution_annotations)

# Check the distribution of speaker IDs in the scenes CSV
label_distribution_scenes = scenes_df['speaker_id'].value_counts()
print("Label Distribution in development_scenes.csv:")
print(label_distribution_scenes)


                        filename       command     start       end
0    1003_speech_false_Licht_aus     Licht aus  12.20090  13.57599
1       1008_speech_true_Ofen_an       Ofen an   6.90112   8.52638
2      1010_speech_true_Radio_an      Radio an  13.03100  14.03146
3  1011_speech_true_Fernseher_an  Fernseher an  14.11030  15.36121
4   1012_speech_true_Heizung_aus   Heizung aus  11.20520  12.70590
                        filename  speaker_id
0         2_speech_true_Ofen_aus         132
1         3_speech_true_Radio_an         132
2         4_speech_true_Alarm_an         132
3        9_speech_true_Radio_aus         132
4  11_speech_false_Fernseher_aus         132
Label Distribution in development_scene_annotations.csv:
command
Licht aus          86
Licht an           78
Heizung an         76
Fernseher an       74
Radio aus          69
Heizung aus        67
Alarm an           66
Radio an           65
Lüftung aus        64
Ofen aus           64
Ofen an            63
Staubsauger an     63

#### Determine CPU/GPU

In [ ]:
def is_gpu_available():
    try:
        import torch
        return torch.cuda.is_available()
    except ImportError:
        pass

    try:
        import tensorflow as tf
        return tf.config.list_physical_devices('GPU') != []
    except ImportError:
        pass

    return False

### Step 1: Data Preprocessing

#### Load Metadata and Annotations


In [42]:
import pandas as pd

# Define file paths
metadata_file = f'{data_dir}/development_scenes.csv'
annotations_file = f'{data_dir}/development_scene_annotations.csv'

# Load the CSV files
metadata = pd.read_csv(metadata_file)
annotations = pd.read_csv(annotations_file)

print("Metadata:")
print(metadata.head())
print("\nAnnotations:")
print(annotations.head())


Metadata:
                        filename  speaker_id
0         2_speech_true_Ofen_aus         132
1         3_speech_true_Radio_an         132
2         4_speech_true_Alarm_an         132
3        9_speech_true_Radio_aus         132
4  11_speech_false_Fernseher_aus         132

Annotations:
                        filename       command     start       end
0    1003_speech_false_Licht_aus     Licht aus  12.20090  13.57599
1       1008_speech_true_Ofen_an       Ofen an   6.90112   8.52638
2      1010_speech_true_Radio_an      Radio an  13.03100  14.03146
3  1011_speech_true_Fernseher_an  Fernseher an  14.11030  15.36121
4   1012_speech_true_Heizung_aus   Heizung aus  11.20520  12.70590


#### Preprocess Audio Files (WAV) and Perform ICA


In [43]:
import os
import librosa
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import FastICA
import numpy as np
from IPython.display import Audio

# Define directories
scenes_wav_dir = f'{data_dir}/scenes/wav'  # Update this path

# Function to load and preprocess audio
def load_and_preprocess_audio(file_path):
    audio, sr = librosa.load(file_path, sr=None)
    scaler = StandardScaler()
    audio = scaler.fit_transform(audio.reshape(-1, 1)).flatten()

    ica = FastICA(n_components=1, whiten='unit-variance')
    audio = ica.fit_transform(audio.reshape(-1, 1)).flatten()

    return audio, sr

# Function to segment audio based on annotations
def segment_audio(audio, sr, start, end):
    start_sample = int(start * sr)
    end_sample = int(end * sr)
    return audio[start_sample:end_sample]

# Preprocess and segment all audio files
preprocessed_segments = []

for idx, row in annotations.iterrows():
    file_name = row['filename']
    command = row['command']
    start = row['start']
    end = row['end']

    file_path = os.path.join(scenes_wav_dir, file_name + '.wav')

    try:
        audio, sr = load_and_preprocess_audio(file_path)
        segment = segment_audio(audio, sr, start, end)
        preprocessed_segments.append((file_name, command, segment, sr))
    except Exception as e:
        print(f"Error processing file {file_path}: {e}")

# Example: display the first segment
print(f"First segment: {preprocessed_segments[0]}")


First segment: ('1003_speech_false_Licht_aus', 'Licht aus', array([-0.08784055, -0.03521105, -0.16523688, ...,  0.15054016,
        0.34248304,  0.2650867 ], dtype=float32), 16000)


In [44]:
import os
import numpy as np
import pandas as pd
import librosa
import soundfile as sf
import random
from IPython.display import Audio
from sklearn.decomposition import FastICA

# Function to apply ICA on audio segments
def apply_ica(segment, sr):
    ica = FastICA(n_components=1, whiten='arbitrary-variance')  # Explicitly set whiten parameter
    segment_reshaped = segment.reshape(-1, 1)
    segment_ica = ica.fit_transform(segment_reshaped).flatten()
    return segment_ica

# Function to preprocess segments and optionally save to the filesystem
def preprocess_and_save_segments(scenes_dir, annotations_path, save_dir=None, save_to_filesystem=False, apply_ica_flag=False):
    # Load the annotations
    annotations_df = pd.read_csv(annotations_path)

    # Ensure the save directory exists if saving to filesystem
    if save_to_filesystem and save_dir is not None:
        os.makedirs(save_dir, exist_ok=True)

    preprocessed_segments = []

    for index, row in annotations_df.iterrows():
        filename = row['filename']
        command = row['command']
        start = row['start']
        end = row['end']

        # Load the audio file
        file_path = os.path.join(scenes_dir, f"{filename}.wav")
        y, sr = librosa.load(file_path, sr=None)

        # Extract the segment
        start_sample = int(start * sr)
        end_sample = int(end * sr)
        segment = y[start_sample:end_sample]

        # Normalize the segment
        segment = librosa.util.normalize(segment)

        # Apply ICA if the flag is set
        if apply_ica_flag:
            segment = apply_ica(segment, sr)

        # Add the segment to the list
        preprocessed_segments.append((filename, command, segment, sr))

        # Save the segment to the filesystem if required
        if save_to_filesystem and save_dir is not None:
            save_path = os.path.join(save_dir, f"{filename}_{start}_{end}.wav")
            sf.write(save_path, segment, sr)

    return preprocessed_segments

# Function to play a random segment from preprocessed segments
def play_random_segment(preprocessed_segments):
    # Select a random segment
    random_segment = random.choice(preprocessed_segments)

    filename, command, audio_data, sample_rate = random_segment

    # Print the command and play the audio segment
    print(f"Filename: {filename}")
    print(f"Command: {command}")

    return Audio(audio_data, rate=sample_rate)

# Function to play a random segment from the filesystem
def play_random_segment_from_filesystem(save_dir, annotations_path):
    # List all the preprocessed segment files
    segment_files = [f for f in os.listdir(save_dir) if f.endswith('.wav')]

    # Select a random segment file
    random_segment_file = random.choice(segment_files)
    random_segment_path = os.path.join(save_dir, random_segment_file)

    # Extract start and end times from the file name
    filename_parts = random_segment_file.split('_')
    filename = '_'.join(filename_parts[:-2])
    start_time = float(filename_parts[-2])
    end_time = float(filename_parts[-1].replace('.wav', ''))

    # Find the command in the annotations
    annotations_df = pd.read_csv(annotations_path)
    command_row = annotations_df[
        (annotations_df['filename'] == filename) &
        (annotations_df['start'] == start_time) &
        (annotations_df['end'] == end_time)
    ]

    if command_row.empty:
        print(f"No matching annotation found for {random_segment_file}")
        return

    command = command_row.iloc[0]['command']

    # Load the audio segment
    y, sr = librosa.load(random_segment_path, sr=None)

    # Print the command and play the audio segment
    print(f"Filename: {filename}")
    print(f"Command: {command}")

    return Audio(y, rate=sr)

# Example usage
scenes_dir = f'{data_dir}/scenes/wav'
annotations_path = f'{data_dir}/development_scene_annotations.csv'
save_dir = f'{data_dir}/scenes/preprocessed_segments'

# Preprocess segments and save to filesystem with optional ICA
preprocessed_segments = preprocess_and_save_segments(scenes_dir, annotations_path, save_dir, save_to_filesystem=True, apply_ica_flag=True)

# Play a random segment from memory
audio_memory = play_random_segment(preprocessed_segments)
display(audio_memory)

# Play a random segment from filesystem
audio_filesystem = play_random_segment_from_filesystem(save_dir, annotations_path)
display(audio_filesystem)


Filename: 215_speech_false_Heizung_aus_Licht_aus
Command: Heizung aus


Filename: 277_speech_false_Licht_aus_Lüftung_an
Command: Lüftung an


### Step 2: Data Augmentation

#### Apply Data Augmentation


In [45]:
import librosa
import numpy as np

# Function to augment audio
def augment_audio(audio, sr):
    augmented = []
    # Original length
    original_length = len(audio)

    # Shift
    shift = int(sr * 0.5)  # shift by 0.5 seconds
    shifted_audio = np.roll(audio, shift)
    augmented.append(shifted_audio)

    # Stretch
    stretch = librosa.effects.time_stretch(audio, rate=0.8)
    if len(stretch) < original_length:
        # Pad with zeros if stretched audio is shorter
        stretch = np.pad(stretch, (0, original_length - len(stretch)), mode='constant')
    else:
        stretch = stretch[:original_length]  # ensure same length
    augmented.append(stretch)

    # Debug information
    for aug_audio in augmented:
        #print(f"Original length: {original_length}, Augmented length: {len(aug_audio)}")
        if len(aug_audio) < 2048:
            print(f"Warning: Augmented segment length {len(aug_audio)} is shorter than n_fft=2048")

    return augmented

# Apply augmentation to all segments
augmented_segments = []
for file_name, command, segment, sr in preprocessed_segments:
    augmented = augment_audio(segment, sr)
    for aug in augmented:
        augmented_segments.append((file_name, command, aug, sr))

# Combine original and augmented data
all_segments = preprocessed_segments + augmented_segments


/usr/local/lib/python3.10/dist-packages/librosa/core/spectrum.py:266: UserWarning: n_fft=2048 is too large for input signal of length=800
  warnings.warn(


### Step 3: Feature Extraction

#### Extract Features with Padding/Truncation and Dynamic `n_fft`


In [46]:
# Function to extract features with dynamic n_fft
def extract_features(segment, sr, max_length):
    n_fft = min(2048, max(256, len(segment) // 2))  # Ensure n_fft is not larger than half the segment length and at least 256
    if len(segment) < n_fft:
        n_fft = len(segment) // 2
    #print(f"Extracting features with n_fft={n_fft} for segment length={len(segment)}")
    hop_length = n_fft // 4  # Typical value for hop_length
    mfccs = librosa.feature.mfcc(y=segment, sr=sr, n_mfcc=13, n_fft=n_fft, hop_length=hop_length)
    mfccs_flat = mfccs.flatten()
    if len(mfccs_flat) < max_length:
        # Pad with zeros
        padded_mfccs = np.pad(mfccs_flat, (0, max_length - len(mfccs_flat)), mode='constant')
        return padded_mfccs
    else:
        # Truncate to max_length
        return mfccs_flat[:max_length]

# Determine the maximum length for padding/truncation
max_length = 0
for _, _, segment, sr in all_segments:
    n_fft = min(2048, max(256, len(segment) // 2))
    if len(segment) < n_fft:
        n_fft = len(segment) // 2
    hop_length = n_fft // 4
    features = librosa.feature.mfcc(y=segment, sr=sr, n_mfcc=13, n_fft=n_fft, hop_length=hop_length).flatten()
    if len(features) > max_length:
        max_length = len(features)

# Extract features for all segments with padding/truncation
feature_data = []
for file_name, command, segment, sr in all_segments:
    features = extract_features(segment, sr, max_length)
    feature_data.append((file_name, command, features))

print(f"Feature extraction completed with max length: {max_length}")


Feature extraction completed with max length: 1170


### Step 4: Model Training and Evaluation

#### Train `GradientBoostingClassifier` with Class Weights and Progress Bar


In [47]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.utils.class_weight import compute_class_weight

# Prepare data for training
X = np.array([features for _, _, features in feature_data])
y = np.array([command for _, command, _ in feature_data])

# Encode string labels to numeric labels
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Split data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

# Compute class weights
classes = np.unique(y_train)
class_weights = compute_class_weight('balanced', classes=classes, y=y_train)
class_weight_dict = {cls: weight for cls, weight in zip(classes, class_weights)}

# Print class weights for debugging
print("Class Weights: ", class_weight_dict)

# Create sample weights
sample_weights = np.array([class_weight_dict[cls] for cls in y_train])

# Print sample weights for debugging
print("Sample Weights: ", sample_weights)


Class Weights:  {0: 0.9740853658536586, 1: 1.065, 2: 0.8729508196721312, 3: 1.298780487804878, 4: 0.9128571428571428, 5: 1.0240384615384615, 6: 0.8192307692307692, 7: 0.7717391304347826, 8: 1.0047169811320755, 9: 1.0579470198675496, 10: 1.0240384615384615, 11: 1.117132867132867, 12: 1.109375, 13: 0.9508928571428571, 14: 1.0721476510067114, 15: 1.2011278195488722}
Sample Weights:  [1.00471698 0.77173913 1.065      ... 1.109375   1.05794702 0.97408537]


In [48]:
# Print class distribution
unique, counts = np.unique(y_train, return_counts=True)
class_distribution = dict(zip(unique, counts))
print("Class Distribution: ", class_distribution)


Class Distribution:  {0: 164, 1: 150, 2: 183, 3: 123, 4: 175, 5: 156, 6: 195, 7: 207, 8: 159, 9: 151, 10: 156, 11: 143, 12: 144, 13: 168, 14: 149, 15: 133}


In [49]:
# Define a plot function for the confusion matrix
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix

def plot_confusion_matrix(y_true, y_pred, labels, title='Confusion Matrix'):
    conf_matrix = confusion_matrix(y_true, y_pred, labels=labels)
    plt.figure(figsize=(10, 8))
    sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=labels, yticklabels=labels)
    plt.title(title)
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.show()


In [50]:
# Train a Gradient Boosting classifier with a progress bar and sample weights
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import classification_report
from tqdm import tqdm

n_estimators = 100
gb = GradientBoostingClassifier(n_estimators=n_estimators, random_state=42, warm_start=True)

# Wrap the training loop with tqdm for the progress bar
for i in tqdm(range(1, n_estimators + 1), desc="Training Gradient Boosting Classifier"):
    gb.n_estimators = i
    gb.fit(X_train, y_train, sample_weight=sample_weights)

# Evaluate the classifier
y_pred_gb = gb.predict(X_val)
print("Gradient Boosting Classifier Report")
print(classification_report(y_val, y_pred_gb, zero_division=0))


Training Gradient Boosting Classifier:  31%|███       | 31/100 [04:48<10:42,  9.32s/it]


KeyboardInterrupt: 

#### Train `RandomForestClassifier` with Class Weights


In [ ]:
# Train a Random Forest classifier with class weights
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

rf = RandomForestClassifier(n_estimators=100, random_state=42, class_weight=class_weight_dict)
rf.fit(X_train, y_train)

# Evaluate the classifier
y_pred_rf = rf.predict(X_val)
print("Random Forest Classifier Report")
print(classification_report(y_val, y_pred_rf, zero_division=0))


### Step 5: Ensemble Classifiers


In [ ]:
from sklearn.ensemble import VotingClassifier

# Combine Random Forest and Gradient Boosting
ensemble = VotingClassifier(estimators=[('rf', rf), ('gb', gb)], voting='soft')
ensemble.fit(X_train, y_train)

# Evaluate the ensemble classifier
y_pred_ensemble = ensemble.predict(X_val)
print("Ensemble Classifier Report")
print(classification_report(y_val, y_pred_ensemble, zero_division=0))

# Plot confusion matrix
abels = np.unique(y_val)
plot_confusion_matrix(y_val, y_pred_ensemble, labels, title='Confusion Matrix - Ensemble')


### Summary

We've consolidated the process into five clear steps:

1. **Data Preprocessing**: Load metadata and preprocess audio files with ICA.
2. **Data Augmentation**: Apply time-shifting and time-stretching to augment the audio data.
3. **Feature Extraction**: Extract MFCC features with appropriate padding/truncation and dynamic `n_fft`.
4. **Model Training and Evaluation**: Train `GradientBoostingClassifier` and `RandomForestClassifier`, handle class weights, and evaluate their performance.
5. **Ensemble Classifiers**: Combine predictions from multiple models to improve robustness.

Let's proceed with running these steps sequentially to ensure there are no errors, warnings, or obstacles. If you encounter any issues or need further adjustments, please let me know!

### Alternative ensembling method: Stacking

In [ ]:
from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import LogisticRegression

# Define base classifiers
estimators = [
    ('rf', RandomForestClassifier(n_estimators=100, random_state=42, class_weight=class_weight_dict)),
    ('gb', GradientBoostingClassifier(n_estimators=100, random_state=42))
]

# Define stacking classifier
stacking_clf = StackingClassifier(estimators=estimators, final_estimator=LogisticRegression())
stacking_clf.fit(X_train, y_train)

# Make predictions
y_pred_stacking = stacking_clf.predict(X_val)

# Evaluate the stacking classifier
print("Stacking Classifier Report")
print(classification_report(y_val, y_pred_stacking, zero_division=0))


In [ ]:
class_weight_dict

### Step-by-Step Guide to Adding a CNN Classifier

1. **Define the CNN Architecture**: We'll use Keras with TensorFlow backend to define the CNN.
2. **Prepare Data for CNN**: Ensure the data is in the right shape and format for the CNN.
3. **Train the CNN**: Train the CNN on the training data.
4. **Evaluate the CNN**: Evaluate its performance on the validation data.
5. **Compare with Other Classifiers**: Compare the CNN's performance with the Gradient Boosting and Random Forest classifiers.

### Step 1: Define the CNN Architecture

We'll use a simple CNN architecture for this example.


In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.utils import to_categorical

# Define the CNN model
def create_cnn_model(input_shape, num_classes):
    model = Sequential([
        Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=input_shape),
        MaxPooling2D(pool_size=(2, 2)),
        Conv2D(64, kernel_size=(3, 3), activation='relu'),
        MaxPooling2D(pool_size=(2, 2)),
        Flatten(),
        Dense(128, activation='relu'),
        Dense(num_classes, activation='softmax')
    ])
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model


### Step 2: Prepare Data for CNN

Ensure the data is reshaped and formatted correctly for the CNN.


In [ ]:
import math

# Convert the feature data to a numpy array
X_cnn = np.array([features for _, _, features in feature_data])
y_cnn = np.array([command for _, command, _ in feature_data])

# Encode string labels to numeric labels
y_cnn_encoded = label_encoder.transform(y_cnn)

# Convert labels to categorical
y_cnn_categorical = to_categorical(y_cnn_encoded)

# Determine input shape for CNN
num_features = X_cnn.shape[1]
# Calculate the dimensions (height and width) such that height * width = num_features
# Here, we aim for the closest possible square dimensions
height = int(math.sqrt(num_features))
width = height
if height * width < num_features:
    width += 1

# Adjust the number of features to match the closest square dimensions
num_adjusted_features = height * width
X_cnn_adjusted = np.zeros((X_cnn.shape[0], num_adjusted_features))
X_cnn_adjusted[:, :num_features] = X_cnn

# Reshape to (num_samples, height, width, 1)
X_cnn_reshaped = X_cnn_adjusted.reshape(-1, height, width, 1)

# Split data into training and validation sets
X_train_cnn, X_val_cnn, y_train_cnn, y_val_cnn = train_test_split(X_cnn_reshaped, y_cnn_categorical, test_size=0.2, random_state=42)

# Print shapes for debugging
print(f"X_cnn shape: {X_cnn.shape}")
print(f"X_cnn_adjusted shape: {X_cnn_adjusted.shape}")
print(f"X_cnn_reshaped shape: {X_cnn_reshaped.shape}")
print(f"X_train_cnn shape: {X_train_cnn.shape}")
print(f"X_val_cnn shape: {X_val_cnn.shape}")


### Step 3: Train the CNN


In [ ]:
# Create the CNN model
input_shape = (height, width, 1)
num_classes = len(np.unique(y_train_cnn.argmax(axis=1)))
cnn_model = create_cnn_model(input_shape, num_classes)

# Train the CNN model
cnn_model.fit(X_train_cnn, y_train_cnn, epochs=10, batch_size=32, validation_data=(X_val_cnn, y_val_cnn))


### Step 4: Evaluate the CNN


In [ ]:
# Evaluate the CNN model
cnn_eval = cnn_model.evaluate(X_val_cnn, y_val_cnn)
print(f"CNN Model Accuracy: {cnn_eval[1]}")

# Make predictions
y_pred_cnn = cnn_model.predict(X_val_cnn)
y_pred_cnn_classes = np.argmax(y_pred_cnn, axis=1)
y_val_cnn_classes = np.argmax(y_val_cnn, axis=1)

# Print classification report
print("CNN Classifier Report")
print(classification_report(y_val_cnn_classes, y_pred_cnn_classes, zero_division=0))

# Plot confusion matrix for CNN
labels = np.unique(y_val_cnn_classes)
plot_confusion_matrix(y_val_cnn_classes, y_pred_cnn_classes, labels, title='Confusion Matrix - CNN')


### Step 5: Compare with Other Classifiers

You already have the evaluation metrics for the Gradient Boosting and Random Forest classifiers. Now you can compare the performance of the CNN with these classifiers.

### Summary

1. **Define CNN Architecture**: Use Keras to define a simple CNN.
2. **Prepare Data**: Reshape the feature data and convert labels to categorical format.
3. **Train CNN**: Train the CNN model on the training data.
4. **Evaluate CNN**: Evaluate the CNN's performance and compare it with other classifiers.

Run the above code step-by-step and compare the results. Let me know if you encounter any issues or need further adjustments!

In [ ]:
from collections import Counter

# Check class distribution in the training set
y_train_labels = label_encoder.inverse_transform(y_train)
class_distribution = Counter(y_train_labels)
print("Class Distribution in Training Set:", class_distribution)

# Visualize class distribution
import matplotlib.pyplot as plt

plt.bar(class_distribution.keys(), class_distribution.values())
plt.xlabel('Class')
plt.ylabel('Frequency')
plt.title('Class Distribution in Training Set')
plt.xticks(rotation=90)
plt.show()


## Using Spectrograms and CNN

1. **Preprocess Audio Data into Spectrograms**: Convert each audio segment into a Mel spectrogram.
2. **Prepare Data for CNN**: Ensure the spectrograms are reshaped and formatted correctly for the CNN.
3. **Define a More Complex CNN**: Use a more complex CNN architecture to handle the spectrogram data.
4. **Train and Evaluate the CNN**: Train the CNN on the training data and evaluate its performance.

### Step 1: Preprocess Audio Data into Spectrograms


In [ ]:
import librosa
import numpy as np

# Function to extract spectrogram features
def extract_spectrogram(segment, sr, n_mels=128, fmax=8000):
    spectrogram = librosa.feature.melspectrogram(y=segment, sr=sr, n_mels=n_mels, fmax=fmax)
    spectrogram_db = librosa.power_to_db(spectrogram, ref=np.max)
    return spectrogram_db

# Apply the spectrogram extraction to all segments
spectrogram_data = []
for file_name, command, segment, sr in all_segments:
    spectrogram = extract_spectrogram(segment, sr)
    spectrogram_data.append((file_name, command, spectrogram))

# Determine the maximum shape for padding
max_shape = max([s.shape for _, _, s in spectrogram_data], key=lambda x: x[1])
print(f"Maximum spectrogram shape: {max_shape}")


### Step 2: Prepare Data for CNN


In [ ]:
# Pad spectrograms to have the same shape
padded_spectrograms = []
for file_name, command, spectrogram in spectrogram_data:
    pad_width = max_shape[1] - spectrogram.shape[1]
    padded_spectrogram = np.pad(spectrogram, ((0, 0), (0, pad_width)), mode='constant')
    padded_spectrograms.append((file_name, command, padded_spectrogram))

# Convert to numpy arrays and reshape
X_spectrogram = np.array([s for _, _, s in padded_spectrograms])
X_spectrogram = X_spectrogram[..., np.newaxis]  # Add channel dimension
y_spectrogram = np.array([command for _, command, _ in padded_spectrograms])

# Encode string labels to numeric labels
y_encoded = label_encoder.transform(y_spectrogram)
y_categorical = to_categorical(y_encoded)

# Split data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_spectrogram, y_categorical, test_size=0.2, random_state=42)

# Print shapes for debugging
print(f"X_train shape: {X_train.shape}")
print(f"X_val shape: {X_val.shape}")
print(f"y_train shape: {y_train.shape}")
print(f"y_val shape: {y_val.shape}")


### Step 3: Define a More Complex CNN


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

# Define a more complex CNN model
def create_complex_cnn_model(input_shape, num_classes):
    model = Sequential([
        Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=input_shape),
        MaxPooling2D(pool_size=(2, 2)),
        Dropout(0.25),
        Conv2D(64, kernel_size=(3, 3), activation='relu'),
        MaxPooling2D(pool_size=(2, 2)),
        Dropout(0.25),
        Conv2D(128, kernel_size=(3, 3), activation='relu'),
        MaxPooling2D(pool_size=(2, 2)),
        Flatten(),
        Dense(256, activation='relu'),
        Dropout(0.5),
        Dense(num_classes, activation='softmax')
    ])
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# Create and train the complex CNN model
input_shape = (X_train.shape[1], X_train.shape[2], X_train.shape[3])
num_classes = y_train.shape[1]
complex_cnn_model = create_complex_cnn_model(input_shape, num_classes)


### Step 4: Train and Evaluate the CNN


In [ ]:
# Train the complex CNN model
complex_cnn_model.fit(X_train, y_train, epochs=20, batch_size=32, validation_data=(X_val, y_val))

# Evaluate the CNN model
cnn_eval = complex_cnn_model.evaluate(X_val, y_val)
print(f"CNN Model Accuracy: {cnn_eval[1]}")

# Make predictions
y_pred_cnn = complex_cnn_model.predict(X_val)
y_pred_cnn_classes = np.argmax(y_pred_cnn, axis=1)
y_val_classes = np.argmax(y_val, axis=1)

# Print classification report
print("CNN Classifier Report")
print(classification_report(y_val_classes, y_pred_cnn_classes, zero_division=0))

# Plot confusion matrix for CNN
plot_confusion_matrix(y_val_classes, y_pred_cnn_classes, labels, title='Confusion Matrix - CNN')


### Summary

1. **Preprocess Audio Data**: Convert audio segments to Mel spectrograms.
2. **Prepare Data**: Pad spectrograms to the same shape and split the data into training and validation sets.
3. **Define CNN**: Use a more complex CNN architecture to handle the spectrogram data.
4. **Train and Evaluate**: Train the CNN and evaluate its performance, then compare with the other classifiers.

Run the above code step-by-step to see how the CNN performs with the spectrogram data. If you encounter any issues or need further adjustments, please let me know!

## Variant: Spectrograms and Conv1D CNN

1. **Prepare Data for Conv1D Model**: Ensure the data is in the right shape and format for the Conv1D model.
2. **Define the Conv1D Model**: Use the provided Conv1D architecture.
3. **Train and Evaluate the Conv1D Model**: Train the Conv1D model on the training data and evaluate its performance.

### Step 1: Prepare Data for Conv1D Model

The Conv1D model expects the data to have the shape `(num_samples, timesteps, features)`. Since we have spectrogram data, we can treat each column of the spectrogram as a timestep.


In [ ]:
import librosa
import numpy as np
from tensorflow.keras.utils import to_categorical

# Function to extract spectrogram features
def extract_spectrogram(segment, sr, n_mels=128, fmax=8000):
    spectrogram = librosa.feature.melspectrogram(y=segment, sr=sr, n_mels=n_mels, fmax=fmax)
    spectrogram_db = librosa.power_to_db(spectrogram, ref=np.max)
    return spectrogram_db

# Apply the spectrogram extraction to all segments
spectrogram_data = []
for file_name, command, segment, sr in all_segments:
    spectrogram = extract_spectrogram(segment, sr)
    spectrogram_data.append((file_name, command, spectrogram))

# Determine the maximum shape for padding
max_shape = max([s.shape for _, _, s in spectrogram_data], key=lambda x: x[1])
print(f"Maximum spectrogram shape: {max_shape}")

# Pad spectrograms to have the same shape
padded_spectrograms = []
for file_name, command, spectrogram in spectrogram_data:
    pad_width = max_shape[1] - spectrogram.shape[1]
    padded_spectrogram = np.pad(spectrogram, ((0, 0), (0, pad_width)), mode='constant')
    padded_spectrograms.append((file_name, command, padded_spectrogram))

# Convert to numpy arrays and reshape
X_spectrogram = np.array([s for _, _, s in padded_spectrograms])
X_spectrogram = X_spectrogram.transpose((0, 2, 1))  # Transpose to (num_samples, timesteps, n_mels)
X_spectrogram = X_spectrogram[..., np.newaxis]  # Add channel dimension
y_spectrogram = np.array([command for _, command, _ in padded_spectrograms])

# Encode string labels to numeric labels
y_encoded = label_encoder.transform(y_spectrogram)
y_categorical = to_categorical(y_encoded)

# Split data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_spectrogram, y_categorical, test_size=0.2, random_state=42)

# Print shapes for debugging
print(f"X_train shape: {X_train.shape}")
print(f"X_val shape: {X_val.shape}")
print(f"y_train shape: {y_train.shape}")
print(f"y_val shape: {y_val.shape}")


### Step 2: Define the Conv1D Model

Use the provided Conv1D architecture:


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout, Input

# Define the Conv1D model with Input layer
input_shape = (X_train.shape[1], X_train.shape[2])

model = Sequential([
    Input(shape=input_shape),  # Use Input layer to define the input shape
    # First Conv1D layer
    Conv1D(filters=8, kernel_size=3, padding='valid', activation='relu', strides=1),
    MaxPooling1D(pool_size=2),
    Dropout(0.3),

    # Second Conv1D layer
    Conv1D(filters=16, kernel_size=3, padding='valid', activation='relu', strides=1),
    MaxPooling1D(pool_size=2),
    Dropout(0.3),

    # Third Conv1D layer
    Conv1D(filters=32, kernel_size=3, padding='valid', activation='relu', strides=1),
    MaxPooling1D(pool_size=2),
    Dropout(0.3),

    # Fourth Conv1D layer
    Conv1D(filters=64, kernel_size=3, padding='valid', activation='relu', strides=1),
    MaxPooling1D(pool_size=2),
    Dropout(0.3),

    # Flatten layer
    Flatten(),

    # Dense Layer 1
    Dense(256, activation='relu'),
    Dropout(0.3),

    # Dense Layer 2
    Dense(128, activation='relu'),
    Dropout(0.3),

    # Output layer
    Dense(num_classes, activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Print model summary for debugging
model.summary()


### Step 3: Train and Evaluate the Conv1D Model


In [ ]:
# Train the Conv1D model
model.fit(X_train, y_train, epochs=20, batch_size=32, validation_data=(X_val, y_val))

# Evaluate the Conv1D model
cnn_eval = model.evaluate(X_val, y_val)
print(f"Conv1D Model Accuracy: {cnn_eval[1]}")

# Make predictions
y_pred_cnn = model.predict(X_val)
y_pred_cnn_classes = np.argmax(y_pred_cnn, axis=1)
y_val_classes = np.argmax(y_val, axis=1)

# Print classification report
print("Conv1D Classifier Report")
print(classification_report(y_val_classes, y_pred_cnn_classes, zero_division=0))

# Plot confusion matrix for Conv1D
plot_confusion_matrix(y_val_classes, y_pred_cnn_classes, labels, title='Confusion Matrix - Conv1D')


### Summary

1. **Prepare Data**: Reshape the spectrogram data for the Conv1D model.
2. **Define Conv1D Model**: Use the provided Conv1D architecture.
3. **Train and Evaluate**: Train the Conv1D model and evaluate its performance.

Run the above code step-by-step to see how the Conv1D model performs with the spectrogram data. If you encounter any issues or need further adjustments, please let me know!

In [ ]:
import pandas as pd

# Load the CSV files
scene_annotations_df = pd.read_csv(f'{data_dir}/development_scene_annotations.csv')
scenes_df = pd.read_csv(f'{data_dir}/development_scenes.csv')

# Check the head of the dataframes to understand their structure
print(scene_annotations_df.head())
print(scenes_df.head())

# Check the distribution of labels in the annotations CSV
label_distribution_annotations = scene_annotations_df['command'].value_counts()
print("Label Distribution in development_scene_annotations.csv:")
print(label_distribution_annotations)

# Check the distribution of speaker IDs in the scenes CSV
label_distribution_scenes = scenes_df['speaker_id'].value_counts()
print("Label Distribution in development_scenes.csv:")
print(label_distribution_scenes)
